In [11]:
# This code has ben run on Google Colab.
# To execute it, you have to move to the right folder.

% cd drive/MyDrive/TaughtNet

[Errno 2] No such file or directory: 'drive/MyDrive/TaughtNet'
/content/drive/MyDrive/TaughtNet


In [12]:
!pip install transformers
!pip install seqeval

## Teachers training

In this example, we will train one teacher for each of the following datasets: *BC2GM*, *BC5CDR-chem*, *NCBI-disease*.

In [ ]:
!python train_teacher.py \
--data_dir 'data/BC2GM' \
--model_name_or_path 'dmis-lab/biobert-base-cased-v1.1' \
--output_dir 'models/Teachers/BC2GM' \
--logging_dir 'models/Teachers/BC2GM' \
--save_steps 10000 

2021-03-29 16:16:06.961537: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Namespace(data_dir='data/BC2GM', do_eval=True, do_predict=True, do_train=True, evaluation_strategy='epoch', logging_dir='models/Teachers/BC2GM', logging_steps=100, max_seq_length=128, model_name_or_path='dmis-lab/biobert-base-cased-v1.1', num_train_epochs=3, output_dir='models/Teachers/BC2GM', per_device_train_batch_size=32, save_steps=10000, seed=1)
03/29/2021 16:16:08 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/29/2021 16:16:08 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=models/Teachers/BC2GM, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, gradient_accumulation_step

In [ ]:
!python train_teacher.py \
--data_dir 'data/BC5CDR-chem' \
--model_name_or_path 'dmis-lab/biobert-base-cased-v1.1' \
--output_dir 'models/Teachers/BC5CDR-chem' \
--logging_dir 'models/Teachers/BC5CDR-chem' \
--save_steps 10000

2021-03-29 16:36:06.082885: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Namespace(data_dir='data/BC5CDR-chem', do_eval=True, do_predict=True, do_train=True, evaluation_strategy='epoch', logging_dir='models/Teachers/BC5CDR-chem', logging_steps=100, max_seq_length=128, model_name_or_path='dmis-lab/biobert-base-cased-v1.1', num_train_epochs=3, output_dir='models/Teachers/BC5CDR-chem', per_device_train_batch_size=32, save_steps=10000, seed=1)
03/29/2021 16:36:07 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/29/2021 16:36:07 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=models/Teachers/BC5CDR-chem, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, gr

In [ ]:
!python train_teacher.py \
--data_dir 'data/NCBI-disease' \
--model_name_or_path 'dmis-lab/biobert-base-cased-v1.1' \
--output_dir 'models/Teachers/NCBI-disease' \
--logging_dir 'models/Teachers/NCBI-disease' \
--save_steps 10000

2021-03-29 16:49:08.663349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Namespace(data_dir='data/NCBI-disease', do_eval=True, do_predict=True, do_train=True, evaluation_strategy='epoch', logging_dir='models/Teachers/NCBI-disease', logging_steps=100, max_seq_length=128, model_name_or_path='dmis-lab/biobert-base-cased-v1.1', num_train_epochs=3, output_dir='models/Teachers/NCBI-disease', per_device_train_batch_size=32, save_steps=10000, seed=1)
03/29/2021 16:49:10 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/29/2021 16:49:10 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=models/Teachers/NCBI-disease, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8

## Global datasets

We need the aggregated datasets for *teachers* in order to obtain their predictions over the whole set of data. Furthermore, we need the aggregated dataset with teachers' labels to train our Student.

In [9]:
!python generate_global_datasets.py

Namespace(data_path='data')
Generating file:  data/GLOBAL/BC2GM/train.tsv
Generating file:  data/GLOBAL/BC5CDR-chem/train.tsv
Generating file:  data/GLOBAL/NCBI-disease/train.tsv
Generating file:  data/GLOBAL/BC2GM/dev.tsv
Generating file:  data/GLOBAL/BC5CDR-chem/dev.tsv
Generating file:  data/GLOBAL/NCBI-disease/dev.tsv
Generating file:  data/GLOBAL/BC2GM/train_dev.tsv
Generating file:  data/GLOBAL/BC5CDR-chem/train_dev.tsv
Generating file:  data/GLOBAL/NCBI-disease/train_dev.tsv
Generating file:  data/GLOBAL/BC2GM/test.tsv
Generating file:  data/GLOBAL/BC5CDR-chem/test.tsv
Generating file:  data/GLOBAL/NCBI-disease/test.tsv
Generating file:  data/GLOBAL/Student/train.tsv
Generating file:  data/GLOBAL/Student/dev.tsv
Generating file:  data/GLOBAL/Student/train_dev.tsv
Generating file:  data/GLOBAL/Student/test.tsv


## Offline generation of teachers' distribution

We obtain the output distribution of each teacher. The $i$-th teacher outputs the probabilities $p_B^i, p_I^i, p_O^i$, $i = \{1,...,k\}$, $k$ being the number of teachers.

We have to aggregate them in a distribution with $2k+1$ labels ($B$ and $O$ for each teacher and the global $O$):

- $P_{Bi} = p_B^i \prod_{j\ne i}{\big(p_I^j + p_O^j\big)}$, 
- $P_{Ii} = p_I^i \prod_{j\ne i}{\big(p_I^j + p_O^j\big)}$, 
- $P_{O} = \prod_i{p_O^i}$

In [10]:
!python generate_teachers_distributions.py \
--data_dir 'data' \
--teachers_dir 'models/Teachers' \
--model_name_or_path 'dmis-lab/biobert-base-cased-v1.1'

Streaming output truncated to the last 5000 lines.

 35% 1326/3823 [02:29<04:42,  8.85it/s]

 35% 1327/3823 [02:29<04:42,  8.83it/s]

 35% 1328/3823 [02:29<04:42,  8.82it/s]

 35% 1329/3823 [02:30<04:42,  8.84it/s]

 35% 1330/3823 [02:30<04:41,  8.84it/s]

 35% 1331/3823 [02:30<04:41,  8.85it/s]

 35% 1332/3823 [02:30<04:41,  8.85it/s]

 35% 1333/3823 [02:30<04:41,  8.83it/s]

 35% 1334/3823 [02:30<04:42,  8.81it/s]

 35% 1335/3823 [02:30<04:42,  8.81it/s]

 35% 1336/3823 [02:30<04:42,  8.82it/s]

 35% 1337/3823 [02:30<04:41,  8.82it/s]

 35% 1338/3823 [02:31<04:41,  8.83it/s]

 35% 1339/3823 [02:31<04:41,  8.83it/s]

 35% 1340/3823 [02:31<04:41,  8.83it/s]

 35% 1341/3823 [02:31<04:41,  8.83it/s]

 35% 1342/3823 [02:31<04:40,  8.84it/s]

 35% 1343/3823 [02:31<04:40,  8.84it/s]

 35% 1344/3823 [02:31<04:40,  8.83it/s]

 35% 1345/3823 [02:31<04:40,  8.82it/s]

 35% 1346/3823 [02:31<04:40,  8.83it/s]

 35% 1347/3823 [02:32<04:40,  8.83it/s]

 35% 1348/3823 [02:32<04:39,  8.84it/s]

 35% 

## Student Training

In [ ]:
!python train_student.py \
--data_dir 'data/GLOBAL/Student' \
--model_name_or_path 'dmis-lab/biobert-base-cased-v1.1' \
--output_dir 'models/Student' \
--logging_dir 'models/Student' \
--save_steps 956 

2021-03-30 15:43:22.368091: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Namespace(data_dir='data/GLOBAL/Student', logging_dir='models/Student', logging_steps=100, max_seq_length=128, model_name_or_path='dmis-lab/biobert-base-cased-v1.1', num_train_epochs=3, output_dir='models/Student', per_device_train_batch_size=32, save_steps=956, seed=1)
03/30/2021 15:43:24 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/30/2021 15:43:24 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=models/Student, overwrite_output_dir=False, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, 